# Updating Archive.org + Excels

## Updating Excel

In [11]:
#searhc for DONE, not DONE and add pages to those that have not yet
from internetarchive import get_session
from internetarchive import modify_metadata
c = {'s3': {'access': 'qJaX9KKXhXkzoN5o', 'secret': 'mmI4XUkxM9O8gZ15'}}
s = get_session(config=c)
query='uploader:"pavelserebrjanyi@gmail.com" AND mediatype:texts'
items=s.search_items(query, fields=["identifier","source_url","imagecount"])


In [31]:
import csv
with open("PrLib_Dataset_DONE overview (0-1920).csv","r", encoding="windows-1251") as csvfile:
    f = csv.reader(csvfile)
    data_file=list(f)

In [20]:
urls=[]
for col in data_file:
    urls.append(col[2])

In [25]:
for item in items:
    if item["source_url"] in urls:
        ind=urls.index(item["source_url"])
        data_file[ind][4]=1
        if data_file[ind][3]=="" and "imagecount" in item:
            
            data_file[ind][3]=item["imagecount"]

In [28]:
import csv
with open("PrLib_Dataset_DONE Pages overview (0-1920).csv","w", encoding="utf-8", newline="") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(data_file)

## Updating years in Archive.org


In [6]:
#Updating years in Archive.org

from internetarchive import get_session
from internetarchive import modify_metadata
c = {'s3': {'access': 'qJaX9KKXhXkzoN5o', 'secret': 'mmI4XUkxM9O8gZ15'}}
s = get_session(config=c)
query='uploader:"pavelserebrjanyi@gmail.com" AND mediatype:texts'
items=s.search_items(query, fields=["identifier","title","imagecount"])

In [7]:
for item in items:
    if "№" in item["title"]:
        print(item["imagecount"])

10
20
9
180
160
240
326
49
110
548
26
28
318
336


KeyError: 'imagecount'

In [6]:
for item in items:
    done=False
    with open("PrLib_Dataset.csv","r", encoding="utf-8") as csvfile:
        writer = csv.DictReader(csvfile)
        for row in writer:
            if row["link"]==item["source_url"]:
                date=int(row["year"])
                modify_metadata(item["identifier"], metadata={'date': date}, access_key ="qJaX9KKXhXkzoN5o", secret_key ='mmI4XUkxM9O8gZ15')  
                done=True
    if not done:
        print("NO YEAR FOUND: ", item["identifier"])
    else:
        print("DONE")

DONE
DONE
NO YEAR FOUND:  addiktivnoepovedenierossijskojmolodezhi87
DONE
DONE
DONE
DONE
DONE
DONE
DONE
NO YEAR FOUND:  atlasistoricheskijhronologicheskijigeog4
DONE
DONE
DONE
DONE
DONE
DONE
DONE
DONE
DONE
DONE
DONE
DONE
NO YEAR FOUND:  chertezhiustrojstvimashin15
DONE
DONE
DONE
DONE
DONE
DONE
DONE
DONE
DONE
DONE
DONE
DONE
DONE
DONE
DONE
DONE
DONE
DONE
DONE
DONE
DONE
DONE
DONE
DONE
DONE
DONE
DONE
DONE
DONE
DONE
DONE
DONE
DONE
DONE
DONE
DONE
NO YEAR FOUND:  entsiklopedijajuridicheskihipoliticheski59
NO YEAR FOUND:  entsiklopedijajuridicheskihpoliticheski44
DONE
NO YEAR FOUND:  evoljutsijatsentralnyhorganovupravlenij17
DONE
DONE
DONE
DONE
NO YEAR FOUND:  fenomenraskolotogosoznanijavrossijskomob84
NO YEAR FOUND:  fenomensotsialnogoparazitizmavsovremenn65
DONE
DONE
NO YEAR FOUND:  filosofijaiuniversitet82
NO YEAR FOUND:  filosofskajateorijaobschestvabnchicher66
NO YEAR FOUND:  formirovanieadaptatsionnogopovedenijanas45
NO YEAR FOUND:  formirovaniemirovozzrenijamolodezhirossi69
NO YEAR FOUND

In [24]:
#CHANGING FROM UTF-8-SIG to utf-8
with open("PrLib_Dataset.csv","r", encoding="utf-8-sig") as csvfile:
    csv_reader = csv.reader(csvfile)
    data1 = [row for row in csv_reader]
with open("PrLIB.csv","w",encoding="utf-8",newline='') as cs:
    writer1=csv.writer(cs)
    writer1.writerows(data1)

# ASyncio scrap


## Final version in data_pages.py:

In [ ]:
from selenium_driverless.types.by import By
from selenium_driverless import webdriver
import asyncio
import csv
all_urls=[]
with open('Prlib_1901-1920.csv', 'r', newline='',encoding="utf-8") as csvfile:
    writer = csv.DictReader(csvfile)
    for row in writer:
        all_urls.append(row["link"])
        

with open("writing_data.txt","r") as file:
    #file.write("\n".join(data))
    data=file.read().splitlines()
import logging
count=len(data)
async def scrape_data_async(url, ind,driver,sem):

    retry=0
    total_retries=2
    while retry<total_retries:
        try:
            async with sem:
                driver1 = await driver.new_window("tab", activate=False)
                await driver1.get(url)
                element = await driver1.find_element(By.ID, 'diva-1-num-pages', timeout=40)
                pages=await element.text
                await driver1.close()
        except:
            time.sleep(1)
            logging.exception("Error")
            if retry==total_retries-1:
                data_entry=(ind,"")
            retry+=1
        else:
            
            data_entry=(ind,pages)
            retry=total_retries

            
    print(ind)
    return data_entry
def write_to_file(resu):
    with open("result_data_pages_intermediary.txt","a") as file:
        count=0
        for item in resu:
            if item=="":
                file.write(str(-1))
            else:
                file.write(str(int(item)))
            
            if count==0:
                file.write(",")
            else:
                file.write("\n")
            count+=1


count=len(data)
import time
start1=time.time()

# list to store the threads
threadList = []

#https://github.com/kaliiiiiiiiii/Selenium-Driverless - TOP
#for each url:
#create Indeces to download

Indeces=[]
with open("result_data_pages_intermediary.txt","r") as file12:
    data=file12.read().splitlines()
    indec=[int(i.split(",")[0]) for i in data]
    for ind in range(len(all_urls)//10):
        if ind not in indec:
            Indeces.append(ind)
    
#sort the indeces + find which ones are missing from all_urls -those to download
#print(Indeces[:50])
# implement to start automatically!
#import sys
#sys.exit()
#Indeces=[0,1,2,3,4,5,6,7,8,9]

lock = asyncio.Lock()
async def main():
    sem=asyncio.Semaphore(10)
    options = webdriver.ChromeOptions()
    options.headless=False
    
    async with webdriver.Chrome(options=options) as driver:
        threads=[]
        for ind in Indeces:
            threads.append(scrape_data_async(all_urls[ind],ind,driver,sem))
            print("Here")
        for coro in asyncio.as_completed(threads):
            result=await coro
            #saving index:
            async with lock:
                write_to_file(result)

import nest_asyncio
#nest_asyncio.apply()
asyncio.run(main())            

print(time.time()-start1)

In [21]:
from selenium_driverless.types.by import By
from selenium_driverless import webdriver
import asyncio
import time
async def scrape_data_async(url, ind,driver,sem):

    retry=0
    total_retries=1
    while retry<total_retries:
        try:
            async with sem:
                driver1 = await driver.new_window("tab")
                await driver1.get(url)
                element = await driver1.find_element(By.ID, 'diva-1-num-pages', timeout=60)
                pages=await element.text
                await driver1.close()
        except:
            time.sleep(1)
            if retry==total_retries-1:
                data_entry=(ind,"")
            retry+=1
        else:
            
            data_entry=(ind,pages)
            retry=total_retries

            
    print(ind)
    return data_entry

In [22]:


import nest_asyncio
start=0
all_urls=["https://www.prlib.ru/item/416405",
"https://www.prlib.ru/item/362502",
"https://www.prlib.ru/item/353787",
"https://www.prlib.ru/item/433180",
"https://www.prlib.ru/item/425847",
         "https://www.prlib.ru/item/443977",
"https://www.prlib.ru/item/443982",
"https://www.prlib.ru/item/443978",
"https://www.prlib.ru/item/443984",
"https://www.prlib.ru/item/1051598",
"https://www.prlib.ru/item/1091167",
"https://www.prlib.ru/item/1091173",
"https://www.prlib.ru/item/426798"]
async def main():
    sem=asyncio.Semaphore(20)
    options = webdriver.ChromeOptions()
    options.add_argument("--headless=new")
    
    async with webdriver.Chrome(options=options) as driver:
        
        threads=[]
        for ind in range(start, len(all_urls)):
            threads.append(scrape_data_async(all_urls[ind],ind,driver,sem))
        for coro in asyncio.as_completed(threads):
            result=await coro
            print(result)

nest_asyncio.apply()
asyncio.run(main())

1
(1, '308')
3
(3, '30')
10
(10, '732')
7
(7, '98')
11
(11, '514')
9
(9, '136')
6
(6, '180')
0
(0, '415')
8
(8, '')
12
5
2
(12, '')
(5, '')
(2, '')
4
(4, '')


In [12]:
resu=(0, '')
with open("test.txt","a") as file:
    count=0
    for item in resu:
        if item=="":
            file.write(str(-1))
        else:
            file.write(str(int(item)))
        if count==0:
            file.write(",")
        else:
            file.write("\n")
        count+=1

0


## Multithreading data pages scrap


In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
#chrome_options = Options()
##chrome_options.add_argument("--headless=new")
#driver = webdriver.Chrome(options=chrome_options)
#driver.get("https://www.prlib.ru/item/323541")

In [2]:
#get all urls to search!
import csv
all_urls=[]
with open('Prlib_1901-1920.csv', 'r', newline='',encoding="utf-8") as csvfile:
    writer = csv.DictReader(csvfile)
    for row in writer:
        all_urls.append(row["link"])

In [7]:
data=[]
count=0

In [20]:
data[-10:]

['', '', '4', '27', '', '', '', '', '', '']

In [22]:
all_urls[200:210]

['https://www.prlib.ru/item/704909',
 'https://www.prlib.ru/item/333504',
 'https://www.prlib.ru/item/891660',
 'https://www.prlib.ru/item/414809',
 'https://www.prlib.ru/item/404546',
 'https://www.prlib.ru/item/704176',
 'https://www.prlib.ru/item/713793',
 'https://www.prlib.ru/item/323927',
 'https://www.prlib.ru/item/439506',
 'https://www.prlib.ru/item/417308']

In [5]:
with open("writing_data.txt","r") as file:
    #file.write("\n".join(data))
    data=file.read().splitlines()

In [6]:
len(data)

489

In [7]:
count=len(data)

In [8]:

def scrape_data(url, ind):
    """
    
    """
chrome_options = Options()
chrome_options.add_argument("--headless=new")
driver1 = webdriver.Chrome(options=chrome_options)
    retry=0
    total_retries=5
    while retry<total_retries:
        try:
    
            driver1.get(url)
            wait = WebDriverWait(driver1, 60)
            element = wait.until(EC.presence_of_element_located((By.ID, 'diva-1-num-pages')))
        except:
            time.sleep(1)
            if retry==total_retries-1:
                data_entry=(ind,"")
            retry+=1
        else:
            pages=element.text
            data_entry=(ind,pages)
            retry=total_retries

            
    print(ind)
    return data_entry

In [36]:
data_full=[None]*len(all_urls)

In [7]:
for ind in range(len(data)):
    data_full=data[ind]

In [9]:
result=[]
count=len(data)

In [ ]:
import time
from concurrent.futures import ThreadPoolExecutor, process, wait
from concurrent.futures import as_completed

start1=time.time()

# list to store the threads
threadList = []

#for each url:
start=count

# initialize the thread pool
with ThreadPoolExecutor(max_workers=15) as executor:
    for ind in range(start, len(all_urls)):
        url=all_urls[ind]
        threadList.append(executor.submit(scrape_data, url,ind))
    for f in as_completed(threadList):
        if f.result():
            result.append(f.result())
# wait for all the threads to complete
  

print(time.time()-start1)

492
499
498
497
494
495
500
501
493
505
512
507
510
509
508
506
511
517
518
515
516
513
520
525
522
523
526
527
524
528
530
531
529
533
532
534
535
538
536
537
496
540
541
503
544
543
489
504
546
547
545
490
548
519
521
514
491
542
539
549
550
552
551
554
553
557
569
555
570
558
571
560
572
573
576
561
577
578
580
581
582
583
584
585
586
587
562
563
565
566
588
567
592
589
568
596
574
597
593
575
595
579
564
602
603
599
601
606
591
611
590
612


Service process refused to terminate gracefully with SIGTERM, escalating to SIGKILL.
Traceback (most recent call last):
  File "C:\Users\Lenovo\Anaconda3\anaconda\Lib\site-packages\urllib3\connectionpool.py", line 459, in _make_request
    httplib_response = conn.getresponse(buffering=True)
                       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
TypeError: HTTPConnection.getresponse() got an unexpected keyword argument 'buffering'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Lenovo\Anaconda3\anaconda\Lib\site-packages\selenium\webdriver\common\service.py", line 184, in _terminate_process
    self.process.wait(60)
  File "C:\Users\Lenovo\Anaconda3\anaconda\Lib\subprocess.py", line 1264, in wait
    return self._wait(timeout=timeout)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Lenovo\Anaconda3\anaconda\Lib\subprocess.py", line 1593, in _wait
    raise TimeoutExpired(self.args, timeout)
subprocess.Tim

615
616


In [ ]:
len(result)

In [ ]:
with open("result_data_pages.txt", "w") as file:
    file.write("\n".join(result))

In [14]:
all_urls[498]

'https://www.prlib.ru/item/461216'

In [12]:
#saving data:
with open("data_pages_1901-1920.txt", "w") as file:
    file.write("\n".join(data))

In [ ]:
#ADD DATA to CSV FILE
import csv

# Step 1: Read the existing CSV file and store its content
csv_file_path = 'Prlib_1901-1920.csv'
with open(csv_file_path, 'r') as file:
    reader = csv.reader(file)
    data_csv = list(reader)

# Step 3: Add the new "City" column header to the first row of the data
data_csv[0].append('pages')

# Step 4: Add the new "City" column values to the remaining rows of the data
for i in range(1, len(data_csv)):
    data_csv[i].append(data[i - 1])

# Step 5: Write the updated data back to the CSV file
with open("Prlib_1901-1920.csv_pages.csv", 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(data_csv)

In [ ]:
import time
start1=time.time()

#for each url:
start=count
for ind in range(start, len(all_urls)):
    retry=0
    total_retries=3
    while retry<total_retries:
        try:
            url=all_urls[ind]
            driver.get(url)
            wait = WebDriverWait(driver, 60)
        except:
            time.sleep(2)
            if retry==total_retries-1:
                data.append("")
            retry+=1
            continue
        try:
            element = wait.until(EC.presence_of_element_located(((By.ID, 'diva-1-num-pages'))))
            pages=element.text
            data.append(pages)
            retry=total_retries
        except:
            if retry<total_retries-1:
                retry+=1
                continue
            else:
                data.append("")
                retry+=1
    count+=1
    print(count)
print(time.time()-start1)